In [17]:
import os
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import obspy
from obspy.core import UTCDateTime
from obspy.signal.filter import envelope
from obspy.signal.invsim import paz_to_freq_resp

In [31]:
!ls ./dataset/SMQ_750103/S14

SMQ_LPX.sac SMQ_LPY.sac SMQ_LPZ.sac SMQ_SPZ.sac


In [44]:
aaa = [1, 2, 3]
bbb = 'aaa'

if 1 in aaa: bbb = 'bbb'
bbb

'bbb'

# クラス・メソッドの定義

In [11]:
class MQ_Analysis:
    '''
    月震の解析用クラス
    
    Attributes:
        station (Int): ステーション番号
        data_lpx (Stream): LP-X DATA
        data_lpy (Stream): LP-Y DATA
        data_lpz (Stream): LP-Z DATA
        data_spz (Stream): SP-Z DATA
    '''
    def __init__(self, input_station):
        '''
        Initialization
        
        Args:
            input_station: ステーション番号
        '''
        station = 0
        data_lpx = None
        data_lpy = None
        data_lpz = None
        data_spz = None
        
        
    def download(self, url):
        '''
        IDを指定してダウンロード (obspyプラグインが必要)
        
        Args:
            url (Str): ダウンロードURL
        '''
        # 末尾の数字に1を足す
        url2 = url.split('.')
        url2[-1] = str(int(aaa.split('.')[-1]) + 1)
        url2 = '.'.join(url2)

        # ダウンロード
        data = obspy.read(url)
        data += obspy.read(url2)
        
        # メンバ変数に格納 (selectが空集合の場合はNoneが入る)
        self.data_lpx = data.select(id='XA.S{}..LPX'.format(self.station))
        self.data_lpy = data.select(id='XA.S{}..LPY'.format(self.station))
        self.data_lpx = data.select(id='XA.S{}..LPZ'.format(self.station))
        self.data_spz = data.select(id='XA.S{}..SPZ'.format(self.station))
        
        
    def read_sac(self, path):
        '''
        SACファイルを読み込む
        
        Args:
            path (Str): SACファイルが入ったディレクトリのパス
        '''
        files = glob(path + '/SMQ_*.sac')
        
        # メンバ変数に格納
        file = path + 'SMQ_LPX.sac'
        if file in files: self.data_lpx = obspy.read(file)
        
        file = path + 'SMQ_LPY.sac'
        if file in files: self.data_lpy = obspy.read(file)
        
        file = path + 'SMQ_LPZ.sac'
        if file in files: self.data_lpz = obspy.read(file)
        
        file = path + 'SMQ_SPZ.sac'
        if file in files: self.data_spz = obspy.read(file)
    
    
    def to_sac(self, path):
        '''
        SACファイルに出力
        
        Args:
            path (Str): SACファイルの出力先のパス
        '''
        # ステーション番号のディレクトリを作成
        dir_path = path + '/S' + str(self.station)
        os.mkdir(dir_path)
        
        # SACファイルの出力
        if self.data_lpx != None:
            self.data_lpx.write(dir_path + '/SMQ_LPX.sac', format='SAC')
        if self.data_lpy != None:
            self.data_lpy.write(dir_path + '/SMQ_LPY.sac', format='SAC')
        if self.data_lpz != None:
            self.data_lpz.write(dir_path + '/SMQ_LPZ.sac', format='SAC')
        if self.data_spz != None:
            self.data_spz.write(dir_path + '/SMQ_SPZ.sac', format='SAC')
        
    
    def data_preprocessing(
        self,
        channel,
        pre_high_freq,
        pre_low_freq,
        post_high_freq,
        post_low_freq,
        paz_remove
    ):
        '''
        前処理用関数

        Args:
            data (Stream): ObspyのStreamオブジェクトに格納されたデータ
            pre_high_freq (Float): Pre-Filtering (Low-Pass) の閾値
            pre_low_freq (Float): Pre-Filtering (High-Pass) の閾値
            post_high_freq (Float): Filtering (Low-Pass) の閾値
            post_low_freq (Float): Filtering (High-Pass) の閾値
            paz_remove (Dict): "poles", "zeros", "gain", "sensitivity"を含むデータ

        Returns:
            data_PSD (Ndarray): dataのPSD
        '''
        data.detrend(type='linear') # detrend
        data.detrend('demean') # demean
        data.taper(0.05, type='cosine')
        data.filter('lowpass', freq=pre_high_freq, zerophase=True)
        data.filter('highpass', freq=pre_low_freq, zerophase=True)
        data.simulate(paz_remove=paz_remove) # Remove response
        data.taper(0.05, type='cosine')
        data.differentiate(method='gradient')
        data_PSD=[0].data
        data.filter('lowpass', freq=freq_high, zerophase=True)
        data.filter('highpass', freq=freq_low, zerophase=True)